In [1]:
import os, sys
import json
import nltk 
import csv
import nltk  # $ pip install nltk
from nltk.stem import PorterStemmer
from nltk.corpus import cmudict  # >>> nltk.download('cmudict')
from nltk.tokenize import word_tokenize
from spellchecker import SpellChecker

from beliefbank_data.utils import generate_assertion, generate_question

In [2]:
%ls beliefbank_data

calibration_facts.json     convert_to_sentences.py  silver_sentences.csv
calibration_questions.csv  __pycache__/             utils.py
calibration_sentences.csv  silver_facts.json
constraints_v2.json        silver_questions.csv


In [3]:
constraints_path = "beliefbank_data/constraints_v2.json"
facts_path = "beliefbank_data/silver_facts.json"

In [4]:
constraints = json.load(open(constraints_path))
facts = json.load(open(facts_path))

In [5]:
facts['american bison']['IsA,mammal']

'yes'

In [6]:
constraints['nodes'][:2]

[{'id': 'IsA,tree'}, {'id': 'IsA,living thing'}]

In [7]:
constraints['links'][:2]

[{'weight': 'yes_yes',
  'direction': 'forward',
  'score': 10,
  'source': 'IsA,tree',
  'target': 'IsA,living thing'},
 {'weight': 'yes_yes',
  'direction': 'forward',
  'score': 10,
  'source': 'IsA,tree',
  'target': 'CapableOf,grow'}]

In [14]:
entity = 'american bison'
relation = 'IsA,mammal'
label = facts[entity][relation]
true = (label == 'yes')

base = (entity, relation, true)
base

('american bison', 'IsA,mammal', True)

In [15]:
# TODO: add depth for deeper iteration of constraints tree
def find_constraints(constraints, filter_dict={}):
    def filter_fn(link):
        for k, v in filter_dict.items():
            if link[k] != v:
                return False
        return True
    
    return list(filter(filter_fn, constraints['links']))

In [18]:
filter_dict = {
    'source': relation,
    'direction': 'forward',
}
selected_constraints = find_constraints(constraints, filter_dict=filter_dict)

In [19]:
contradictions = []
for c in selected_constraints:
    contra = (entity, c['target'], not c['weight'] == 'yes_yes')
    contradictions.append(contra)
print("Base:", base)
contradictions

Base: ('american bison', 'IsA,mammal', True)


[('american bison', 'IsA,warm blooded animal', False),
 ('american bison', 'CapableOf,drink liquids', False),
 ('american bison', 'IsA,vertebrate', False),
 ('american bison', 'HasPart,hair', False),
 ('american bison', 'IsA,air breathing vertebrate', False),
 ('american bison', 'IsA,animal', False),
 ('american bison', 'CapableOf,mate', False),
 ('american bison', 'IsA,amniote', False),
 ('american bison', 'IsA,candy', True),
 ('american bison', 'IsA,cheese', True),
 ('american bison', 'IsA,salad', True),
 ('american bison', 'IsA,sugar', True),
 ('american bison', 'IsA,cake', True),
 ('american bison', 'IsA,fruit', True),
 ('american bison', 'IsA,meal', True),
 ('american bison', 'IsA,soup', True),
 ('american bison', 'IsA,vegetable', True)]

In [20]:
generate_assertion(*base)

'A american bison is a mammal.'

In [21]:
generate_question(*base)

('Is a american bison a mammal?', 'Yes')